# **Ex3 - Finding a subset of extremely important attributes**

---



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
import seaborn as sn
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
import collections

Bring the data from sklearn, normalize and split it:

In [ ]:
X_digits, y_digits = load_digits(return_X_y=True)

n_samples, n_features = X_digits.shape
n_digits = len(np.unique(y_digits))

#print("n_digits: %d, \t n_samples %d, \t n_features %d" %(n_digits, n_samples, n_features))

# normalize
x_matrix_hat = scale(X_digits)

# split data
x_train, x_test, y_train, y_test = train_test_split(x_matrix_hat, y_digits, test_size=0.3, random_state=42)


Get Best C (Constant):

In [ ]:
c = [0.01, 0.1, 1, 10, 100, 1000, 10000]
acc = []
best_acc = 0
best_c = 0
i = 0
for i in range(len(c)):
    object_lr = LogisticRegression(max_iter=1000, C=c[i], penalty='l2').fit(x_train, y_train)
    acc.append(object_lr.score(x_test, y_test))
    if acc[i] > best_acc:
        best_acc = acc[i]
        best_c = c[i]
c = best_c

Greedy algorithm:

In [ ]:
f = [] # features

while len(f) < 5:  # run until we have got 5 features
    max_score = 0  # keeps the highest score
    feature_max_score = None  # keeps the feature with the highest score
    for i in range(n_features):  # for each feature
        if i in f:  # if we chose it already to f, skip
            continue
       
        temp = f.copy()
        temp.append(i)
        object_lr = LogisticRegression(max_iter=1000, C=c, penalty='l2').fit(x_train[:, temp], y_train) # penalty l2 means we use norma 2
        y_test = y_test.astype('int')
        score = object_lr.score(x_test[:, temp], y_test)  # calculate score (accuracy on the given test)
        
        if max_score < score:  # if it's higher than the max score update max score
            max_score = score
            feature_max_score = i
    f.append(feature_max_score)  # add the feature the got the highest score

# object_lr = LogisticRegression(max_iter=1000, C=c, penalty='l2').fit(x_train[:, f], y_train)
# prediction = object_lr.predict(x_test[:, f])  # predict with the features we found

#greedy_pred = prediction

print("Greedy Algorithm - Five most important features: {}".format(f))

21
42
36
26
61
Greedy Algorithm - Five most important features: [21, 42, 36, 26, 61]


MI algotithm:

In [ ]:
f = [] # features
eleventosixteen = fivetoten = zerotofour = 0
arr_y = [0] * 10  #ten options 

for i in range(n_features):  # for each feature
  if 0 <= i <= 4:
    zerotofour += 1
  if 5 <= i <= 10:
    fivetoten += 1
  if 11 <= i <= 16:
    eleventosixteen += 1

X_new = SelectKBest(chi2, k=5).fit_transform(X_digits, y_digits) # using probability and MI alg
 
# checking which colunms did we get in x_new
X_digits_T = X_digits.T
X_new_T = X_new.T
compare = lambda x, y: collections.Counter(x) == collections.Counter(y)

def get_col_num(col):
  for i in range(len(X_digits_T)):
   
    if compare(X_digits_T[i], col):
      return i

for i in range(len(X_new_T)):
  f.append(get_col_num(X_new_T[i]))

print("MI Algorithm - Five most important features: {}".format(f))

MI Algorithm - Five most important features: [33, 34, 42, 43, 54]


**Analysing the results:**

The greedy algorithm takes the most important and meaningful features, so the results are the accurate as possible.
The MI (mutual information) algorithm takes the features with the highest importability of y (the results).

The results above show that:

The greedy algorithm chose the features with the highest corelation  between the input and the output of the test (the first feature that added is number 21), than the algorithm chose the next feature regarding its first choice (what feature resulting the most accurate importability of y, by adding it to the calculation within the chosen feature[s]?). It seems like the algorithm passes on many combinations of features, that way we might miss a better combination of features.

The MI algorithm chose the features with the highest corelation between the input and the output (of the test), means these features have higher probabilities than the other that the algorithm didn't choose eventually.


By testing the results, we can learn that the MI algorithm is more accurate, because the greedy algorithm passes on many features' combinations that might be better, while probabilities are the same (doesn't matter what the algorithm chose before).